In [2]:
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 
from langgraph.types import Command 
from langchain_core.messages import AIMessage


load_dotenv("../.env")

# 1. Setup Database and LLM
db_name = "banking_customers_indian.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_name}")
llm = ChatOpenAI(model="gpt-4o") # Using gpt-4o for stability

# 2. Setup Toolkit and Tools
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

# 3. System Prompt
prompt_template = """
You are an expert banking assistant. 
Follow these steps for every query:
1. List tables to see what is available.
2. Query the schema of relevant tables.
3. Execute the SQL query.

If a query for 'HDFC' returns no results, try 'HDFC Bank' or check unique values in the `bank_name` column.
Limit results to {top_k}.
"""
system_prompt = prompt_template.format(top_k=5)

# 4. Initialize Agent with Middleware
agent = create_agent(
    llm,
    tools,
    system_prompt=system_prompt,
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"sql_db_query": True}, 
        ), 
    ], 
    checkpointer=InMemorySaver(), 
)

# 5. Execution Loop
query = "What is the average monthly balance of HDFC Bank customers?"
config = {"configurable": {"thread_id": "unique_thread_123"}} 

print("--- STARTING AGENT ---")
for step in agent.stream({"messages": [{"role": "user", "content": query}]}, config, stream_mode="values"):
    if "__interrupt__" in step: 
        print("\n[INTERRUPT]: Agent is waiting for approval to run SQL...")
        # Automatically resume for this script, but in a real UI you would wait for a button click
        break 
    if "messages" in step:
        step["messages"][-1].pretty_print()



--- STARTING AGENT ---
================================ Human Message =================================

What is the average monthly balance of HDFC Bank customers?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_rBsQlAVh6zjqEy4mAjugxB1h)
 Call ID: call_rBsQlAVh6zjqEy4mAjugxB1h
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

consumers
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_OgXbGxlvYKTMNPwSi8uhDN9J)
 Call ID: call_OgXbGxlvYKTMNPwSi8uhDN9J
  Args:
    table_names: consumers
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE consumers (
	id INTEGER, 
	customer_name TEXT, 
	account_number TEXT, 
	bank_name TEXT, 
	monthly_average_balance REAL, 
	yearly_income REAL, 
	place TEXT, 
	top_spending_category TEXT, 
	PRIM

In [3]:
print("\n--- APPROVING SQL EXECUTION ---")
# Resume execution with approval
for step in agent.stream(
    Command(resume={"decisions": [{"type": "approve"}]}), 
    config,
    stream_mode="values",
):
    if "messages" in step:
        msg = step["messages"][-1]
        msg.pretty_print() # This works for ALL message types
        
        # To capture the final answer specifically:
        # Check if it's an AIMessage and if it has no tool calls
        if isinstance(msg, AIMessage) and not msg.tool_calls and msg.content:
            print(f"\nFINAL ANSWER FOUND: {msg.content}")


--- APPROVING SQL EXECUTION ---
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_EYofkpTncb486FLTzAaibxiP)
 Call ID: call_EYofkpTncb486FLTzAaibxiP
  Args:
    query: SELECT AVG(monthly_average_balance) AS average_balance FROM consumers WHERE bank_name = 'HDFC Bank';
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_EYofkpTncb486FLTzAaibxiP)
 Call ID: call_EYofkpTncb486FLTzAaibxiP
  Args:
    query: SELECT AVG(monthly_average_balance) AS average_balance FROM consumers WHERE bank_name = 'HDFC Bank';
================================= Tool Message =================================
Name: sql_db_query

[(100027.21222222222,)]
================================== Ai Message ==================================

The average monthly balance of HDFC Bank customers is approximately 100,027.21.

FINAL ANSWER FOUND: The average monthly balance of HDFC Bank customers is appro